# 16. Identify possible map coverage errors (required) (sequence diagram not required)
- Identify the geographic area defined as the difference between the geographic area covered by the
state and the geographic area covered by the union of all precincts in the state. 
- Store the associated data in the errors data structure for subsequent addition to the DB. 
- The regions so identified are potential ghost precincts. 

In [1]:
!ls

DetectGhost.ipynb               fixPrecinctNum.ipynb
VA_Precinct_Cleaned.GeoJSON.zip stateDif.json
VA_Precinct_Cleaned.json        statePrecinctdif.json
backup:real source              vaprecincts2016.json


In [2]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
import time

In [3]:
precinctGDF = gpd.read_file("VA_Precinct_Cleaned.json")

In [4]:
# drop some unneeded columns
c = list(precinctGDF.columns)
c.remove("Precinct #")
c.remove("geometry")
precinctGDF.drop(c, inplace=True, axis=1)
precinctGDF.head(1)

,Precinct #,geometry
0,601,"POLYGON ((-75.66233 37.77053, -75.66244 37.770..."


In [5]:
# drop some unneeded columns
stateGDF = gpd.read_file("../State/VA_State.json")
c = list(stateGDF.columns)
c.remove("geometry")
stateGDF.drop(c, inplace=True, axis=1)
stateGDFcopy = stateGDF.copy()
#stateGDF = stateGDF.to_crs("EPSG:3857")

# get difference of state and state with buffer

In [6]:
stateEnlarged = stateGDFcopy.to_crs("EPSG:3857").unary_union.buffer(60.96)
stateEnlarged = gpd.GeoDataFrame({"geometry":stateEnlarged}, crs="EPSG:3857")
stateEnlarged = stateEnlarged.to_crs("EPSG:4326")
stateDif = gpd.overlay(stateEnlarged, stateGDF, how="difference")
stateDif.to_file("stateDif.json", driver="GeoJSON")

# filter out small dots

In [7]:
# shape with area < 0.00001 will be dropped
area_threshold = 0.00001
for index, row in precinctGDF.iterrows():
    cur_geometry = row["geometry"]
    polygon_list = []
    if cur_geometry.geom_type == "MultiPolygon":
        for shape in cur_geometry:
            if shape.area >= area_threshold:
                polygon_list.append(shape)
        if len(polygon_list) == 1:
            precinctGDF.at[index, "new_geo"] = polygon_list[0]
        else:
            new_multipoly = MultiPolygon(polygon_list)
            precinctGDF.at[index, "new_geo"] = new_multipoly
    else:
        precinctGDF.at[index, "new_geo"] = cur_geometry
precinctGDF.set_geometry("new_geo", inplace=True, drop=True)

# get difference of precinct and regular state

In [9]:
stateGDF = stateGDFcopy.copy()

In [10]:
# fix invalid polygons
precinctGDF["new"] = ""
for index, row in precinctGDF.iterrows():
    if not precinctGDF.at[index, "geometry"].is_valid: 
        precinctGDF.at[index, "new"] = row["geometry"].buffer(0.0)
    else:
        precinctGDF.at[index, "new"] = row["geometry"]
precinctGDF.set_geometry("new", inplace=True, drop=True)

In [11]:
# check any invalid left
for index, row in precinctGDF.iterrows():
    if not precinctGDF.at[index, "geometry"].is_valid: 
        print(precinctGDF.at[index, "geometry"].is_valid)

In [ ]:
start = time.time()
union_poly = ""
counter = 0
for index, row in precinctGDF.iterrows():
    if union_poly == "":
        union_poly = row["geometry"]
    else:
        union_poly = union_poly.union(row["geometry"])
    print("#", end="")
    if counter %15 == 0:
        print("{}%".format((counter/len(precinctGDF))*100))
    counter += 1
#precinctGDF = precinctGDF.unary_union
end = time.time()
print("Total time: {} minutes".format((end-start)/60))
# precinctGDF = precinctGDF.buffer(60.96)
# gdf = precinctGDF.copy()

#0.0%
###############0.6132461161079313%
###############1.2264922322158627%
###############1.839738348323794%
###############2.4529844644317254%
###############3.066230580539657%
###############3.679476696647588%
###############4.292722812755519%
###############4.905968928863451%
###############5.519215044971382%
###############6.132461161079314%
###############6.745707277187245%
###############7.358953393295176%
###############7.972199509403106%
###############8.585445625511039%
###############9.198691741618969%
###############9.811937857726901%
###############10.425183973834832%
###############11.038430089942764%
###

In [ ]:
gdf = gpd.GeoDataFrame({"geometry": precinctGDF})

In [ ]:
# statePrecinctdif = gpd.overlay(stateGDF, gdf, how="difference")
statePrecinctdif = gpd.overlay(stateGDF, precinctGDF, how="difference")

In [ ]:
statePrecinctdifCopy = statePrecinctdif.copy()

In [ ]:
statePrecinctdif.to_file("statePrecinctdif.json", driver="GeoJSON")

In [ ]:
statePrecinctdif.head()

# turn all multipolygon into polygon

In [13]:
newgdf = gpd.GeoDataFrame(crs="EPSG:4326")
newgdf["geometry"] = ""
newgdf["index"] = 0

In [14]:
i = 0
for index, row in statePrecinctdif.iterrows():
    g = row["geometry"]
    if g.geom_type == "MultiPolygon":
        for shape in g:
            newgdf.at[i, "geometry"] = shape
            newgdf.at[i, "index"] = i
            i += 1
    else:
        newgdf.at[i, "geometry"] = g
        newgdf.at[i, "index"] = i
        i += 1
newgdf.set_geometry("geometry", inplace=True)

In [15]:
newgdf.head(1)

,geometry,index
0,"POLYGON ((-93.51492 31.02570, -93.52231 31.020...",0.0


In [16]:
newgdf.to_file("statePrecinctdif.json", driver="GeoJSON")

# filter by area

In [ ]:
# dif.to_file("RI_Ghost.json", driver="GeoJSON")

In [18]:
newgdf = newgdf.to_crs("EPSG:3857")

In [19]:
a = 3716.1216
newgdf["filtered"] = ""
for index, row in newgdf.iterrows():
    if row.geometry.geom_type == "MultiPolygon":
        m = MultiPolygon([shape for shape in row["geometry"] if shape.area >= a])
        if m.is_empty:
            newgdf.drop(index, inplace=True)
        else:
            newgdf.at[index, "filtered"] = m
    else:
        if row["geometry"].area < a:
            newgdf.drop(index, inplace=True)
        else:
            newgdf.at[index, "filtered"] = row["geometry"]

In [20]:
newgdf.set_geometry("filtered", drop=True, inplace=True)

In [21]:
newgdf = newgdf.to_crs("EPSG:4326")

In [22]:
newgdf.to_file("statePrecinctdif.json", driver="GeoJSON")

# drop precincts that is fully inside stateDif
- stateDif is the difference between og state and state buffered by 200feet

In [52]:
for index, row in stateDif.iterrows():
    index_drop = newgdf[newgdf["geometry"].within(stateDif.loc[index]["geometry"])].index.to_list()
    for i in index_drop:
        newgdf.drop(index=i, inplace=True)
        print("#", end="")

# delete unwanted polygons

In [53]:
def deletePoly(gdf, index):
    gdf.drop(index=index, inplace=True)
    return gdf

In [20]:
# index_to_delete = [46, 43, 44, 45, 39, 34, 33, 27, 26, 23, 14, 35, 38, 40, 42]
for i in index_to_delete:
    newgdf = deletePoly(newgdf, i)

# create key for ghost table

In [54]:
newgdf.head(1)

,geometry,index
0,"POLYGON ((-93.51492 31.02570, -93.52231 31.020...",0.0


In [55]:
newgdf["id"] = ""
i = 0
for index, row in newgdf.iterrows():
    newgdf.at[index, "id"] = "tx-ghost-" + str(i)
    i += 1
newgdf.head(2)

,geometry,index,id
0,"POLYGON ((-93.51492 31.02570, -93.52231 31.020...",0.0,tx-ghost-0
10,"POLYGON ((-93.52855 31.07060, -93.52896 31.069...",10.0,tx-ghost-1


In [24]:
newgdf.to_file("RI_Ghost.json", driver="GeoJSON")